In [49]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [50]:
nodeset = pd.read_csv(f"../out/features.csv")
df = pd.read_csv(f"../out/edges.csv")

In [51]:
nodeset["label"] = nodeset["node"].str.contains(r"(sbox|mixcolumn)",
                                          case=False, na=False).astype(int)
nodeset

/var/folders/fc/rq9v0gn94dq1cpr9d1jydxq40000gn/T/ipykernel_39774/453769152.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  nodeset["label"] = nodeset["node"].str.contains(r"(sbox|mixcolumn)",


,node,Degree,Hamming distance,Node,Paths,and,mux,node_number,or,xor,label
0,325:AS,18,3607,"325:AS\nadd_roundkey[95:64] = { m_col[0][1], m...",0,0,1,0,0,1,0
1,InvMixColumn1.729:AS,1,0,"InvMixColumn1.729:AS\nm2c = mul4x8({ 4'hd }, s...",0,0,0,1,0,1,1
2,188:AS,9,8747,188:AS\nnext_key[63:32] = w[2] ^ next_key[95:64];,1,0,1,2,0,1,0
3,93:AL,3,0,93:AL,510,0,0,3,0,0,0
4,209:IF,3,0,209:IF,0,0,0,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
222,326:AS,18,3659,"326:AS\nadd_roundkey[63:32] = { m_col[0][2], m...",0,0,1,222,0,1,0
223,156:AS,2,0,156:AS\ntext_out = (enc_dec == 1'b0)? { state[...,0,0,1,223,0,0,0
224,147:NS,2,0,147:NS\nkey_val <= key_val;,31,0,0,224,0,0,0
225,344:IF,3,0,344:IF,0,0,0,225,0,0,0


In [52]:
if "label" in nodeset.columns:
    label = "label"
else:
    label = "xor" # suppose xor is the label
# In the paper, nodes belonging to the Sbox and Mixcolumns modules are labeled as 1

class_values = sorted(nodeset[label].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(nodeset["node_number"].unique()))}

nodeset["node_number"] = nodeset["node_number"].apply(lambda name: paper_idx[name])
df["source"] = df["source"].apply(lambda name: paper_idx[name])
df["target"] = df["target"].apply(lambda name: paper_idx[name])
nodeset[label] = nodeset[label].apply(lambda value: class_idx[value])

print(nodeset.label.value_counts())

label
0    195
1     32
Name: count, dtype: int64


In [53]:
train_data = nodeset.iloc[0:196]
test_data = nodeset.iloc[196:]

In [54]:
hidden_units = [32, 32]
learning_rate = 0.0001
dropout_rate = 0.3
num_epochs = 32
batch_size = 20

In [55]:
def run_experiment(model, x_train, y_train):
    # Compile the model.
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[keras.metrics.BinaryAccuracy(name="acc"),
keras.metrics.Precision(), keras.metrics.Recall()],
    )
    # Create an early stopping callback.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.10,
        callbacks=[early_stopping],
    )

    return history

In [56]:
def display_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history["loss"])
    ax1.plot(history.history["val_loss"])
    ax1.legend(["train", "test"], loc="upper right")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")

    ax2.plot(history.history["acc"])
    ax2.plot(history.history["val_acc"])
    ax2.legend(["train", "test"], loc="upper right")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    plt.show()

In [57]:
def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.relu))

    return keras.Sequential(fnn_layers, name=name)

In [58]:
feature_names = set(nodeset.columns) - {"node_number", "Node", "node", label}
num_features = len(feature_names)
num_classes = len(class_idx)

print(train_data[list(feature_names)])
# Create train and test features as a numpy array.
x_train = train_data[list(feature_names)].to_numpy()
x_test = test_data[list(feature_names)].to_numpy()
#print(x_train)
# Create train and test targets as a numpy array.
y_train = train_data[label]
y_test = test_data[label]

     mux  xor  Paths  Hamming distance  Degree  or  and
0      1    1      0              3607      18   0    0
1      0    1      0                 0       1   0    0
2      1    1      1              8747       9   0    0
3      0    0    510                 0       3   0    0
4      0    0      0                 0       3   0    0
..   ...  ...    ...               ...     ...  ..  ...
191    0    0      0                 0       8   0    0
192    0    0    374                 0       3   0    0
193    0    0      0                 0       4   0    0
194    0    0      0                 0       8   0    0
195    0    1      0                 0       4   0    0

[196 rows x 7 columns]


In [59]:
def create_baseline_model(hidden_units, num_classes, dropout_rate=0.2):
    inputs = layers.Input(shape=(num_features,), name="input_features")
    x = create_ffn(hidden_units, dropout_rate, name=f"ffn_block1")(inputs)
    for block_idx in range(4):
        # Create an FFN block.
        x1 = create_ffn(hidden_units, dropout_rate, name=f"ffn_block{block_idx + 2}")(x)
        # Add skip connection.
        x = layers.Add(name=f"skip_connection{block_idx + 2}")([x, x1])
    # Compute logits.
    logits = layers.Dense(num_classes-1, name="logits")(x)
    # Create the model.
    return keras.Model(inputs=inputs, outputs=logits, name="baseline")


baseline_model = create_baseline_model(hidden_units, num_classes, dropout_rate)
baseline_model.summary()

history = run_experiment(baseline_model, x_train, y_train)

Model: "baseline"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block1          │ (None, 32)        │      1,468 │ input_features[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block2          │ (None, 32)        │      2,368 │ ffn_block1[0][0]  │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection2    │ (None, 32)        │          0 │ ffn_block1[0][0], │
│ (Add)               │                   │            │ ffn_block2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block3          │ (None, 32)        │      2,368 │ skip_connection2… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection3    │ (None, 32)        │          0 │ skip_connection2… │
│ (Add)               │                   │            │ ffn_block3[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block4          │ (None, 32)        │      2,368 │ skip_connection3… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection4    │ (None, 32)        │          0 │ skip_connection3… │
│ (Add)               │                   │            │ ffn_block4[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block5          │ (None, 32)        │      2,368 │ skip_connection4… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection5    │ (None, 32)        │          0 │ skip_connection4… │
│ (Add)               │                   │            │ ffn_block5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ logits (Dense)      │ (None, 1)         │         33 │ skip_connection5… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,973 (42.86 KB)

 Trainable params: 10,383 (40.56 KB)

 Non-trainable params: 590 (2.30 KB)

Epoch 1/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - acc: 0.8646 - loss: 2.1822 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_acc: 0.8500 - val_loss: 2.4177 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 2/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.8728 - loss: 2.0507 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_acc: 0.8500 - val_loss: 2.4177 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 3/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.8751 - loss: 2.0132 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_acc: 0.8500 - val_loss: 2.4177 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 4/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.8510 - loss: 2.4021 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_acc: 0.8500 - val_loss: 2.4177 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 5/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.8690 - loss: 2.1113 - precision_4: 0.0000e+00 - recall_4: 0.00

In [60]:
# Create an edges array (sparse adjacency matrix) of shape [2, num_edges].
edges = df[["source", "target"]].to_numpy().T
edge_weights = tf.ones(shape=edges.shape[1])

node_features = tf.cast(
    nodeset.sort_values("node_number")[list(feature_names)].to_numpy(), dtype=tf.dtypes.float32
)
# Create graph with node features, edges, and edge_weights.
graph_info = (node_features, edges, edge_weights)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)
graph_info

Edges shape: (2, 556)
Nodes shape: (227, 7)


(<tf.Tensor: shape=(227, 7), dtype=float32, numpy=
 array([[ 1.,  1.,  0., ..., 18.,  0.,  0.],
        [ 0.,  1.,  0., ...,  1.,  0.,  0.],
        [ 1.,  1.,  1., ...,  9.,  0.,  0.],
        ...,
        [ 0.,  0., 31., ...,  2.,  0.,  0.],
        [ 0.,  0.,  0., ...,  3.,  0.,  0.],
        [ 0.,  1.,  0., ...,  1.,  0.,  0.]], dtype=float32)>,
 array([[220,  44, 191, ..., 157,  84, 180],
        [112,   7,  56, ..., 215, 148,  60]]),
 <tf.Tensor: shape=(556,), dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,

In [61]:
class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.3,
        aggregation_type="mean",
        combination_type="concat",
        normalize=False,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        if self.combination_type == "gated":
            self.update_fn = layers.GRU(
                units=hidden_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                dropout=dropout_rate,
                return_state=True,
                recurrent_dropout=dropout_rate,
            )
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations, weights=None):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        if weights is not None:
            messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_repesentations):

        num_nodes = node_repesentations.shape[0]
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")

        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):

        if self.combination_type == "gru":
            # Create a sequence of two elements for the GRU layer.
            h = tf.stack([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            # Concatenate the node_repesentations and aggregated_messages.
            h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            # Add node_repesentations and aggregated_messages.
            h = node_repesentations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):

        node_repesentations, edges, edge_weights = inputs

        node_indices, neighbour_indices = edges[0], edges[1]
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)

        aggregated_messages = self.aggregate(
            node_indices, neighbour_messages, node_repesentations
        )

        return self.update(node_repesentations, aggregated_messages)

In [62]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.3,
        normalize=True,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create the second GraphConv layer.
        self.conv2 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        self.compute_logits = layers.Dense(units=num_classes,activation="softmax", name="logits")

    def call(self, input_node_indices):
        # Preprocess the node_features to produce node representations.
        x = self.preprocess(self.node_features)
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x1 + x
        # Apply the second graph conv layer.
        x2 = self.conv2((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x2 + x
        # Postprocess node embedding.
        x = self.postprocess(x)
        # Fetch node embeddings for the input node_indices.
        node_embeddings = tf.gather(x, input_node_indices)
        print(node_embeddings)
        # Compute logits
        return self.compute_logits(node_embeddings)

In [63]:
gnn_model = GNNNodeClassifier(
    graph_info=graph_info,
    num_classes=num_classes,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

print("GNN output shape:", gnn_model(tf.constant([[1, 10, 100]], dtype=tf.int32)))

gnn_model.summary()

Tensor("GatherV2:0", shape=(1, 3, 32), dtype=float32)
tf.Tensor(
[[[2.40869612e-01 2.36483052e-01 2.74317712e-01 0.00000000e+00
   3.42208564e-01 2.71773875e-01 1.10088274e-01 0.00000000e+00
   4.92725568e-03 0.00000000e+00 2.20477268e-01 3.14979367e-02
   7.34976172e-01 0.00000000e+00 0.00000000e+00 2.33531430e-01
   0.00000000e+00 0.00000000e+00 1.78394601e-01 2.20100224e-01
   0.00000000e+00 1.98388457e-01 1.38119385e-02 1.81935951e-01
   0.00000000e+00 1.81236900e-02 0.00000000e+00 0.00000000e+00
   3.46033096e-01 0.00000000e+00 0.00000000e+00 1.04800351e-01]
  [2.67213249e+00 0.00000000e+00 5.13039351e+00 1.88325548e+00
   4.01517153e-02 3.57394433e+00 3.02160692e+00 0.00000000e+00
   1.33438826e+00 0.00000000e+00 2.31742311e+00 0.00000000e+00
   9.93183804e+00 1.05250752e+00 0.00000000e+00 7.69066954e+00
   6.55350626e-01 1.66266596e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 8.11348259e-01 0.00000000e+00 1.47569585e+00
   0.00000000e+00 6.27542317e-01 0.00000000e+00 0.00

Model: "gnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ preprocess (Sequential)         │ (227, 32)              │         1,468 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ graph_conv1 (GraphConvLayer)    │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ graph_conv2 (GraphConvLayer)    │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ postprocess (Sequential)        │ (227, 32)              │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (1, 3, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,678 (61.24 KB)

 Trainable params: 14,832 (57.94 KB)

 Non-trainable params: 846 (3.30 KB)

In [64]:
y_train1 = tf.keras.utils.to_categorical(
    y_train, num_classes=2)
y_test1 = tf.keras.utils.to_categorical(
    y_test, num_classes=2)

In [65]:
x_train = train_data.node_number.to_numpy()
history = run_experiment(gnn_model, x_train, y_train1)

Epoch 1/32
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
1/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - acc: 0.8500 - loss: 0.4606 - precision_5: 0.8500 - recall_5: 0.8500Tensor("gnn_model_1/GatherV2:0", shape=(20, 32), dtype=float32)
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.8449 - loss: 1.1013 - precision_5: 0.8449 - recall_5: 0.8449 - val_acc: 0.8500 - val_loss: 1.3259 - val_precision_5: 0.8500 - val_recall_5: 0.8500
Epoch 2/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.8501 - loss: 1.0585 - precision_5: 0.8501 - recall_5: 0.8501 - val_acc: 0.8500 - val_loss: 0.8185 - val_precision_5: 0.8500 - val_recall_5: 0.8500
Epoch 3/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.8385 - loss: 0.5663 - precision_5: 0.8385 - recall_5: 0.8385 - val_acc: 0.8500 - val_loss: 0.4280 - val_precision_5: 0.8500 - val_recall_5: 0.8500
Epoch 4/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.8632 - loss: 0.3056 - precis

In [66]:
gnn_model.save_weights("../out/saved_weights.weights.h5")

In [67]:
model = GNNNodeClassifier(
    graph_info=graph_info,
    num_classes=num_classes,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

model.build(input_shape=(None, graph_info[0].shape[1]))

model.load_weights('../out/saved_weights.weights.h5')

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'gnn_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [68]:
from keras import backend as K
model.compile()
x_test = test_data.node_number.to_numpy()
_, test_accuracy, precision, recall = gnn_model.evaluate(x=x_test, y=y_test1, verbose=0)
print(f"Test accuracy: {round(test_accuracy * 100, 2)}%")
print(f"Test precision: {(precision* 100)}%")
print(f"Test recall: {(recall * 100)}%")

Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Test accuracy: 93.55%
Test precision: 93.54838728904724%
Test recall: 93.54838728904724%


In [69]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_pred= gnn_model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_testt = np.argmax(y_test1, axis=1)
cm = confusion_matrix(y_testt, y_pred)
print(cm)
print(classification_report(y_testt,y_pred))

Tensor("gnn_model_1/GatherV2:0", shape=(31, 32), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
[[27  0]
 [ 2  2]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        27
           1       1.00      0.50      0.67         4

    accuracy                           0.94        31
   macro avg       0.97      0.75      0.82        31
weighted avg       0.94      0.94      0.93        31

